In [1]:
# workhorse modules
import pandas as pd
from datetime import timedelta, datetime
from pathlib import Path
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import HTML
import matplotlib.pyplot as plt


# local utility functions
from utils.general_utils import (
    add_fiscal_year,
    compute_is_on_time,
    set_pd_params,
    choose_pms_or_cms,
    compute_pm_cm,
    compute_pm_cm_by_month,
    compute_kpi_table,
    compute_kpi_table_by_month,
)
# local visualization functions
from utils.vis_utils import set_plot_params, pointplot_with_barplot
plt.ioff()

In [2]:
set_pd_params()
set_plot_params(width=12, height=6)
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
DATA_URL = 'https://gist.githubusercontent.com/chriddyp/cb5392c35661370d95f300086accea51/raw/8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/indicators.csv'


EXPLANATION = """\
<div class="app-sidebar">
<p><em>Compare different development indicators.</em><p>

<p>Select what indicators to plot in the dropdowns, and use the slider
to sub-select a fraction of years to include in the plot.</p>

<p>Data and idea copied from the <a href="https://dash.plot.ly/getting-started-part-2">
Plotly Dash documentation</a>.</p>

<p>This example demonstrates combining matplotlib with Jupyter widgets. For more interactive plots,
consider using <a href="https://github.com/bloomberg/bqplot">bqplot</a>.
</div>
"""

In [4]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [5]:
class App:
    def __init__(self, df):
        self._df = df
        fy_choices = ["Requested", "Completed", "Closed"]
        type_choices = ["PMs", "CMs", "All WRs"]
        self._fy_dropdown = self._create_dropdown(fy_choices)
        self._type_dropdown = self._create_dropdown(type_choices)
        self._plot_container = widgets.Output()

        _app_container = widgets.VBox(
            [
                widgets.HBox([self._fy_dropdown, self._type_dropdown]),
                self._plot_container,
            ],
            layout=widgets.Layout(align_items="center", flex="3 0 auto"),
        )
        self.container = widgets.VBox(
            [
                widgets.HTML(
                    (
                        "<h1>Development indicators</h1>"
                        '<h2 class="app-subtitle"><a href="https://github.com/pbugnion/voila-gallery/blob/master/country-indicators/index.ipynb">Link to code</a></h2>'
                    ),
                    layout=widgets.Layout(margin="0 0 5em 0"),
                ),
                widgets.HBox(
                    [
                        _app_container,
                        widgets.HTML(
                            EXPLANATION, layout=widgets.Layout(margin="0 0 0 2em")
                        ),
                    ]
                ),
            ],
            layout=widgets.Layout(
                flex="1 1 auto", margin="0 auto 0 auto", max_width="1024px"
            ),
        )
        self._update_app()

    @classmethod
    def from_pickle(cls):
        scorecard_path = Path.cwd() / "data" / "scorecard_data.pkl"
        df = pd.read_pickle(scorecard_path)
        print(f"The work request data contains {len(df):,} rows.")
        return cls(df)

    def _create_dropdown(self, options, initial_index=0):
        dropdown = widgets.Dropdown(options=options, value=options[initial_index])
        dropdown.observe(self._on_change, names=["value"])
        return dropdown

    def _create_plot(self, fy_option, type_option):
        df = self._df
        print(fy_option)
        df = add_fiscal_year(self._df, assign_fy_on=fy_option)
        df = choose_pms_or_cms(df, selection=type_option)
        
        compliance = compute_kpi_table(
            df,
            f"percent_{type_option}_on_time",
            f"total_{type_option}",
            grouping_var="fiscal_year",
        )

        pointplot_with_barplot(
            compliance,
            x=compliance.index,
            point_y=f"percent_{type_option}_on_time",
            bar_y=f"total_{type_option}",
            xlabel=f"Fiscal Year {fy_option}",
            ylabel_point=f"Percent {type_option} On Time (Red line)",
            ylabel_bar=f"Total {type_option} (Grey bars)",
            title=f"Percent of {type_option} Completed On Time",
        )

    def _on_change(self, _):
        self._update_app()

    def _update_app(self):
        fy_option = self._fy_dropdown.value
        type_option = self._type_dropdown.value

        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(fy_option, type_option)
            plt.show()

In [6]:
app = App.from_pickle()

app.container


The work request data contains 108,105 rows.
